In [2]:
import helpers as hp
# import Preprocessing - Separating Recipes as prep
from preprocess import pos_tag_db, lemmatize_db
import nltk
from itertools import permutations, chain
from collections import Counter
from nltk.metrics import BigramAssocMeasures
from math import log
import pandas as pd

0	EACH PORTION:  1 Apple with 2 Tbsp Syrup (5Â½ oz)

1	PAN SIZE:  18X26-inch Sheet pan

2	TEMPERATURE: 325Â°F Oven

3	Score cored, unpeeled apples once around middle to prevent bursting.  Place apples on pans.

4	Mix sugar, cinnamon, and salt thoroughly.

5	Combine with water and butter or margarine.  Pour 1Â½ qt syrup over apples in each pan.

6	Bake 45 minutes or until tender, basting occasionally.

7	Serve each apple with 2 tbsp. (1-AA ladel) syrup.

8	1.  In Step 1, 40 lb fresh apples A.P. will yield 33 lb 14 oz cored apples.

9	2.  In Step 4, if convection oven is used, bake at 325Â° F for 30 minutes or until tender.  Use low fan, closed vent.

0	[('each', 'NOUN'), ('portion', 'NOUN'), ('1', 'NUM'), ('apple', 'NOUN'), ('with', 'ADP'), ('2', 'NUM'), ('tbsp', 'NOUN'), ('syrup', 'NOUN'), ('5â½', 'NOUN'), ('oz', 'NOUN')]
1	[('pan', 'NOUN'), ('size', 'NOUN'), ('18x26-inch', 'ADJ'), ('sheet', 'NOUN'), ('pan', 'NOUN')]
2	[('325â°f', 'NUM'), ('oven', 'NOUN')]
3	[('score', 'ADV'), ('cored'

In [10]:
ingredients2 = a = pd.read_csv("data/nyt-ingredients-snapshot-2015.csv")["name"]

In [12]:
ingredients2 = set(ingredients2)

In [13]:
ingredients2

{nan,
 'pimenton',
 'cocoa powder',
 'plum jam',
 'garni',
 'pine liqueur',
 'scallions, thinly sliced, dark green parts separated',
 'glucose syrup',
 'chili pepper sauce',
 'medium shallot',
 'white supermarket bread (1 or 2 loaves)',
 'Minty yogurt chutney',
 'vegan dark chocolate',
 'toasted slivered almonds, chopped candied orange peel or candied ginger, optional',
 'roasted, unsalted peanuts',
 'hard boiled eggs',
 'white Cloud Ear fungus',
 'minced candied ginger',
 'live blue crabs',
 'Salsa cruda',
 'English pastry',
 'Puligny Montrachet',
 'Ground nutmeg',
 'basil vinegar',
 'kale leaves',
 'small tomato, chopped',
 'zest of 1 lemon',
 'Truffle',
 'light cream',
 'star anise',
 'wheat flour, or as needed',
 'the rind of 1 lemon from which the juice has been squeezed',
 'Tabasco sauce to taste',
 'dukkah',
 'black Nice olives',
 'Herbs de Provence',
 'Rock salt',
 'tuna, 1 1/2-inch-thick',
 'Applesauce with horseradish',
 'Mexican hot sauce',
 'demiglace',
 'toasted walnut hal

In [ ]:
# co occurrences 

spansize = 5 # symmetric ±5w span

cooccs_surface = Counter()

for recipe in lemmatized: # lemmatized? not sure it's the right one
    for i,w in enumerate(recipe):
        if w.split("-")[-1] in ["ingre"]: # ingredients is the target vectors 
            span_range = list(range(max(i - spansize, 0), i)) # left side indices (range, then list so we can extend)
            span_range.extend(range(i + 1, min(i + spansize + 1, len(recipe)))) # extend by right side indices
            for cw in [recipe[idx] for idx in span_range]:
                cooccs_surface[(w, cw)] += 1

In [ ]:
cooccs_surface.most_common(20) # print 20 most common co-occurrences

In [ ]:
# filtering least and most frequent lemmas 

# least frequent
lemmas_frequencies = Counter(chain(*lemmatized))
print("least common lemmas:", list(reversed(lemmas_frequencies.most_common()))[:20])

# most frequent 
print("most common lemmas:", lemmas_frequencies.most_common(20))

In [ ]:
filtered_lemmas_frequencies = Counter()
selected_ingre_frequencies = Counter()

for k,v in lemmas_frequencies.items():
    pos = k.split("-")[-1]
    if v >= 10 and \
    pos in ["NN", "NNS", "NNP", "NNPS", "JJ", "JJR", "JJS", "VB", "VBD", "VBG", "VBN", "VBP", "VBZ", "IN", "CD", "ingre"]
        filtered_lemmas_frequencies[k] = v
        if pos == "ingre" : # targeting all ingredients
            selected_ingre_frequencies[k] = v

print(">>", len(filtered_lemmas_frequencies), "context lemmas")
print(filtered_lemmas_frequencies.most_common(20))
print(">>", len(selected_ingre_frequencies), "target nouns")

In [ ]:
# filtered co-occurrences

filtered_cooccs_surface = Counter()

for k,v in cooccs_surface.items():
    if k[0] in selected_ingre_frequencies and k[1] in filtered_lemmas_frequencies:
        filtered_cooccs_surface[k] = v

In [ ]:
filtered_cooccs_surface.most_common(20) # print 20 filteres most common co-occurrences

In [ ]:
# vector representations 

# Measure of choice - sppmi

def sppmi(o_11, r_1, c_1, all_freqs_sum, n):
    """Smoothed Positive Pointwise Mutual Information"""
    return ppmi(o_11, r_1, (c_1**0.75)/all_freqs_sum, n)

sppmi_surface = Counter()

N = sum(cooccs_surface.values())  # note that this is NOT the reduced dictionary
s = sum([v**0.75 for v in reuters_lemmas_frequencies.values()]) # calculating Pa(c)

for k,v in filtered_cooccs_surface.items():
    sppmi_surface[k] = sppmi(v, lemmas_frequencies[k[0]], lemmas_frequencies[k[1]], s, N) 

In [ ]:
sppmi_surface.most_common(15) 